In [56]:
import pandas as pd
import numpy as np

In [57]:
test_data=pd.read_csv('Test.csv')

In [58]:
test_data['iuniqueid']=test_data['uniqueid'] + 'x' + test_data['country']

In [59]:
#drop columns
columns_to_drop=['year','country','uniqueid']
new_data=test_data.drop(columns=columns_to_drop)

In [60]:
new_data.rename({'iuniqueid':'uniqueid'},inplace=True)
new_data

,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type,iuniqueid
0,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government,uniqueid_6056xKenya
1,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private,uniqueid_6060xKenya
2,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent,uniqueid_6065xKenya
3,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent,uniqueid_6072xKenya
4,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent,uniqueid_6073xKenya
...,...,...,...,...,...,...,...,...,...,...
10081,Rural,No,2,62,Female,Spouse,Married/Living together,Primary education,Self employed,uniqueid_2998xUganda
10082,Urban,Yes,8,42,Male,Head of Household,Married/Living together,Primary education,Self employed,uniqueid_2999xUganda
10083,Urban,Yes,1,39,Male,Head of Household,Single/Never Married,Secondary education,Other Income,uniqueid_3000xUganda
10084,Rural,Yes,6,28,Female,Spouse,Married/Living together,Primary education,Self employed,uniqueid_3001xUganda


In [61]:
new_data.set_index('iuniqueid', inplace=True)

In [62]:
new_data.columns

Index(['location_type', 'cellphone_access', 'household_size',
       'age_of_respondent', 'gender_of_respondent', 'relationship_with_head',
       'marital_status', 'education_level', 'job_type'],
      dtype='object')

In [63]:
high_cardiniallity_categorical_columns=[ 'relationship_with_head',
       'marital_status', 'education_level', 'job_type','location_type','cellphone_access','gender_of_respondent']
numerical_columns=['household_size','age_of_respondent']

In [64]:
new_data['job_type'].value_counts()

job_type
Self employed                   2773
Informally employed             2372
Farming and Fishing             2370
Remittance Dependent            1107
Formally employed Private        444
Other Income                     423
No Income                        276
Formally employed Government     166
Government Dependent             100
Dont Know/Refuse to answer        55
Name: count, dtype: int64

In [65]:
new_data['education_level'].value_counts()

education_level
Primary education                  5479
Secondary education                1902
No formal education                1836
Tertiary education                  492
Vocational/Specialised training     359
Other/Dont know/RTA                  18
Name: count, dtype: int64

In [66]:
new_data['marital_status'].value_counts()

marital_status
Married/Living together    4663
Single/Never Married       3447
Widowed                    1107
Divorced/Seperated          868
Dont know                     1
Name: count, dtype: int64

In [67]:
new_data['gender_of_respondent'].value_counts()

gender_of_respondent
Female    5847
Male      4239
Name: count, dtype: int64

In [68]:
new_data['location_type'].value_counts()

location_type
Rural    6189
Urban    3897
Name: count, dtype: int64

In [69]:
new_data['cellphone_access'].value_counts()

cellphone_access
Yes    7559
No     2527
Name: count, dtype: int64

In [70]:
new_data['relationship_with_head'].value_counts()

relationship_with_head
Head of Household      5487
Spouse                 2845
Child                   962
Parent                  465
Other relative          258
Other non-relatives      69
Name: count, dtype: int64

In [71]:
new_data['location_type'].value_counts()

location_type
Rural    6189
Urban    3897
Name: count, dtype: int64

In [72]:
new_data.isna().sum()

location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64

In [73]:
new_data.duplicated().sum()

1614

In [74]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import joblib
#Load transformers for unseen data
loaded_scaler = joblib.load('scaler.joblib')

In [75]:
unseen_data=new_data

In [76]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import joblib   
    
#Encoding high cardiniallity categorical columns
unseen_data=pd.get_dummies(unseen_data,columns=high_cardiniallity_categorical_columns,drop_first=True)


#Scaling numerical columns
unseen_data[numerical_columns]=loaded_scaler.transform(unseen_data[numerical_columns])




In [77]:
from xgboost import XGBClassifier
import joblib

# Load the saved XGBoost model
loaded_xgb_model = XGBClassifier()
loaded_xgb_model.load_model(r'C:\Users\hp\Victor-Files\Financial_Inclusion\xgb_model_saved.model')

In [78]:
#Make predictions on the test set
y_pred = loaded_xgb_model.predict(unseen_data)



In [79]:
# Create submission DataFrame
submission = pd.DataFrame({"uniqueid": test_data["uniqueid"] + " x " + test_data["country"],
                           "bank_account": y_pred})

In [80]:
# Save the DataFrame to a CSV file
submission.to_csv('predicted_values_xg.csv', index=False)

In [81]:
submission

,uniqueid,bank_account
0,uniqueid_6056 x Kenya,1
1,uniqueid_6060 x Kenya,1
2,uniqueid_6065 x Kenya,0
3,uniqueid_6072 x Kenya,0
4,uniqueid_6073 x Kenya,0
...,...,...
10081,uniqueid_2998 x Uganda,0
10082,uniqueid_2999 x Uganda,0
10083,uniqueid_3000 x Uganda,0
10084,uniqueid_3001 x Uganda,0
